In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# 1. Hyperparameters
batch_size = 1
learning_rate = 0.001
num_epochs = 10

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 2. Data Preprocessing: No resizing, keep original 32x32 size
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

# 3. Load CIFAR-10 Dataset
full_train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#full_test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 4. Filter Dataset for Cats and Dogs
def filter_cats_dogs(dataset):
    targets = torch.tensor(dataset.targets)  # Convert labels to a tensor
    mask = (targets == 3) | (targets == 5)  # Keep only labels 3 (cat) and 5 (dog)
    dataset.targets = targets[mask].tolist()  # Update targets
    dataset.data = dataset.data[mask.numpy()]  # Update data
    return dataset

train_dataset = filter_cats_dogs(full_train_dataset)
#test_dataset = filter_cats_dogs(full_test_dataset)

# 5. Update Labels: Map [3, 5] -> [0, 1]
def remap_labels(dataset):
    dataset.targets = [0 if label == 3 else 1 for label in dataset.targets]  # Cat = 0, Dog = 1
    return dataset

train_dataset = remap_labels(train_dataset)
#test_dataset = remap_labels(test_dataset)

# 6. Data Loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# RPQ Function
def rpq(input_vector, rows, columns):
    #sign_quantization
    def quantize(signature):
        for i in range(0,columns) :
            if signature[i] < 0:
                signature[i] = 1
            else:
                signature[i] = 0
        return signature

    random_rpq_matrix = torch.randn(rows, columns, device=device)  # Move RPQ matrix to GPU

    # Flatten 32 * 32 to 1 * 1024
    flattened_vector = input_vector.view(-1)

    # Dot product of input vector and R
    signature = torch.matmul(flattened_vector, random_rpq_matrix)

    # Quantization -> sign-based
    signature_quantized = quantize(signature)
    return signature_quantized

# MCache
mcache = {}
cache_hits = 0
cache_misses = 0
binary_key = ''

# 7. Define a Simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: cat and dog

    def forward(self, x):
        global cache_hits, binary_key, cache_misses
        if binary_key in mcache:
            # Already first layer output is available
            cache_hits += 1
            x = mcache[binary_key].detach()
        else:
            cache_misses += 1
            x = self.pool(torch.relu(self.conv1(x)))
            mcache[binary_key] = x.detach()
            
        # Second layer is continued normally now
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)  # Move model to GPU

# 8. Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 9. Training Loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move images and labels to GPU

        # Generate RPQ signature and binary key
        rpq_signature_output = rpq(images, 1024, 20) # 1024 coz 32 * 32 (right now) , 20 columns coz signature length 

        # Convert the binary tensor to a string representation of binary
        binary_key = ''.join(map(str, rpq_signature_output.tolist()))  # tensor to list and then to string

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")
print("Cache_hits :", cache_hits)
print("Cache_misses:", cache_misses)

Using device: cuda
Files already downloaded and verified
Epoch 1/10
Epoch 1/10, Loss: 0.6580
Epoch 2/10
Epoch 2/10, Loss: 0.5963
Epoch 3/10
Epoch 3/10, Loss: 0.5687
Epoch 4/10
Epoch 4/10, Loss: 0.5463
Epoch 5/10
Epoch 5/10, Loss: 0.5209
Epoch 6/10
Epoch 6/10, Loss: 0.5000
Epoch 7/10
Epoch 7/10, Loss: 0.4764
Epoch 8/10
Epoch 8/10, Loss: 0.4569
Epoch 9/10
Epoch 9/10, Loss: 0.4337
Epoch 10/10
Epoch 10/10, Loss: 0.4197
Training complete!
Cache_hits : 4591
Cache_misses: 95409


In [48]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# 1. Hyperparameters
batch_size = 1
learning_rate = 0.001
num_epochs = 10

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 2. Data Preprocessing: No resizing, keep original 32x32 size
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

# 3. Load CIFAR-10 Dataset
full_train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#full_test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 4. Filter Dataset for Cats and Dogs
def filter_cats_dogs(dataset):
    targets = torch.tensor(dataset.targets)  # Convert labels to a tensor
    mask = (targets == 3) | (targets == 5)  # Keep only labels 3 (cat) and 5 (dog)
    dataset.targets = targets[mask].tolist()  # Update targets
    dataset.data = dataset.data[mask.numpy()]  # Update data
    return dataset

train_dataset = filter_cats_dogs(full_train_dataset)
#test_dataset = filter_cats_dogs(full_test_dataset)

# 5. Update Labels: Map [3, 5] -> [0, 1]
def remap_labels(dataset):
    dataset.targets = [0 if label == 3 else 1 for label in dataset.targets]  # Cat = 0, Dog = 1
    return dataset

train_dataset = remap_labels(train_dataset)
#test_dataset = remap_labels(test_dataset)

# 6. Data Loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

####################################################################################################################

outputs_final = {}
total_cache_hits = 0
total_cache_misses = 0

# RPQ Function
def rpq(input_vector, rows, columns):
    #sign_quantization
    def quantize(signature):
        for i in range(0,columns) :
            if signature[i] < 0:
                signature[i] = 1
            else:
                signature[i] = 0
        return signature

    random_rpq_matrix = torch.randn(rows, columns, device=device)  # Move RPQ matrix to GPU

    # Flatten 32 * 32 to 1 * 1024
    flattened_vector = input_vector.view(-1)

    # Dot product of input vector and R
    signature = torch.matmul(flattened_vector, random_rpq_matrix)

    # Quantization -> sign-based
    signature_quantized = quantize(signature)
    return signature_quantized


def mcache_func(binary_key, mcache):
    if binary_key in mcache:
        return True
    else:
        return False

# 7. Define a Simple CNN
class SimpleCNN_with_RPQ_Layer(nn.Module):
    def __init__(self):
        super(SimpleCNN_with_RPQ_Layer, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: cat and dog
        
        self.mcache = {}
        self.cache_hits = 0
        self.cache_misses = 0 
        self.binary_key = ''
    
    def reset(self):
        self.mcache = {}
        self.cache_hits = 0
        self.cache_misses = 0 
        self.binary_key = ''
        print("Mcache Reset")
        
    def forward(self):
        global total_cache_hits, total_cache_misses
        #reset mcache, cache_hits and cache_misses
        self.reset()
        
        # first conv layer
        weights_conv1 = {}
        weights_conv2 = {}
        input_label_GPU = {}
        i = 0 
        #
        for input_image, input_label in train_loader:
            input_image, input_label_GPU = input_image.to(device), input_label.to(device) # Move image, labels to GPU

            # Generate RPQ signature and binary key
            rpq_signature_output = rpq(input_image, 1024, 20) # 1024 coz 32 * 32 (right now) , 20 columns coz signature length 
            #print("Signature:")
            
            # Convert the binary tensor to a string representation of binary
            self.binary_key = ''.join(map(str, rpq_signature_output.tolist()))  # tensor to list and then to string
            
            if mcache_func(self.binary_key,self.mcache) == True :
                self.cache_hits += 1
                weights_conv1[i] = self.mcache[self.binary_key]
            else :
                self.cache_misses +=1
                weights_conv1[i] = self.pool(torch.relu(self.conv1(input_image))) 
                self.mcache[self.binary_key] = weights_conv1[i]
            i += 1
            
        print("First Layer Training Complete")
        total_cache_hits += self.cache_hits
        total_cache_misses += self.cache_misses
        print("First layer->Cache_hits", self.cache_hits)
        print("First Layer->Cache_misses", self.cache_misses)
        
        #reset mcache, cache_hits and cache_misses
        self.reset()
        
        # Second layer is continued using the output of the First conv layer (weights_conv1[]) 
        j = 0
        for i in range(0,len(weights_conv1)):
            # Generate RPQ signature and binary key
            rpq_signature_output = rpq(input_image, 1024, 20) # 1024 coz 32 * 32 (right now) , 20 columns coz signature length 

            # Convert the binary tensor to a string representation of binary
            self.binary_key = ''.join(map(str, rpq_signature_output.tolist()))  # tensor to list and then to string
            
            if mcache_func(self.binary_key,self.mcache) == True :
                self.cache_hits += 1
                weights_conv2[j] = self.mcache[i][self.binary_key]
            else :
                self.cache_misses += 1
                weights_conv2[j] = self.pool(torch.relu(self.conv2(weights_conv1[i]))) 
                self.mcache[i][self.binary_key] = weights_conv2[i]
            j += 1
            

        print("Second layer Training Complete")            
        total_cache_hits += self.cache_hits
        total_cache_misses += self.cache_misses
        print("Second layer->Cache_hits", self.cache_hits)
        print("Second Layer->Cache_misses", self.cache_misses)
        
        #The other layers(Pool,FC1,FC2) is continued using the outputs of the Second conv layer (weights_conv2[])
        j = 0
        for i in range(0,len(weights_conv2)):
            outputs_final[j] = weights_conv2[i].view(-1, 32 * 8 * 8)
            outputs_final[j] = torch.relu(self.fc1(outputs_final[j]))
            outputs_final[j] = self.fc2(outputs_final[j])
            j += 1

model = SimpleCNN_with_RPQ_Layer().to(device)  # Move model to GPU


######################################################################################################################

# 8. Training Loop 

# Forward Propogation
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model()

#Backward Propogation

        
print("Training complete!")
print("Average_Cache_hits :", total_cache_hits/10)
print("Average_Cache_misses:", total_cache_misses/10)

Using device: cuda
Files already downloaded and verified
Epoch 1/10
Mcache Reset
First Layer Training Complete
First layer->Cache_hits 48
First Layer->Cache_misses 9952
Mcache Reset
Second layer Training Complete
Second layer->Cache_hits 45
Second Layer->Cache_misses 9955
Epoch 2/10
Mcache Reset
First Layer Training Complete
First layer->Cache_hits 48
First Layer->Cache_misses 9952
Mcache Reset
Second layer Training Complete
Second layer->Cache_hits 41
Second Layer->Cache_misses 9959
Epoch 3/10
Mcache Reset
First Layer Training Complete
First layer->Cache_hits 57
First Layer->Cache_misses 9943
Mcache Reset
Second layer Training Complete
Second layer->Cache_hits 64
Second Layer->Cache_misses 9936
Epoch 4/10
Mcache Reset
First Layer Training Complete
First layer->Cache_hits 43
First Layer->Cache_misses 9957
Mcache Reset
Second layer Training Complete
Second layer->Cache_hits 39
Second Layer->Cache_misses 9961
Epoch 5/10
Mcache Reset
First Layer Training Complete
First layer->Cache_hits 4

In [ ]:

# 8. Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# loss calculation and Bqckward Propogation  
    ##########################################################    
        loss = criterion(outputs, input_label)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")



# 10. Test the Model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")